# Tutorial 2: Interacting with Databases in Sessions

In this tutorial we will create and then interact with a *relational* SQLite database, using `sqlalchemy`'s `Session` object.

We will:

* Create a relational database containing two linked tables: "Objects" (with ID, ra and dec) and "Distances" (with ID and distance).
* Retrieve data into numpy arrays so that it can be used For Science.

This tutorial was adapted for astronomy from [the one at `pythoncentral.io`](http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/).

### Requirements

You will need to have `sqlalchemy` installed.
```
pip install sqlalchemy
```

In [1]:
import sqlalchemy as sq
import os
import numpy as np
import random

In the previous tutorial, we used a somewhat manual approach where we separately defined a Table in a database, a mapper that would connect a python object to the database, and the python object itself. This is tedious.

In this tutorial, we will use an alternative approach: declaratives. The advantage of using declaratives is that they allow us to define `Table`, mappers and python objects at once in a class definition. Let's see.

In [2]:
from sqlalchemy.ext.declarative import declarative_base

## Creating Database

Initialize a declarative.

In [3]:
Base = declarative_base()

Now, we create Python class objects for two linked tables: "Objects" and "Distances". When these classes are defined (or imported), a database table is initialized for each one (if it has not been already).

In [4]:
class Object(Base):
    __tablename__ = 'object'
    
    # Define columns like we did in the last notebook
    id = sq.Column(sq.Integer, primary_key=True)
    ra = sq.Column(sq.Float, nullable=False)
    dec = sq.Column(sq.Float, nullable=False)

    
class Distance(Base):
    __tablename__ = 'distance'
    
    # Again define columns like above. These columns
    # are normal python instance attributes.
    id = sq.Column(sq.Integer, primary_key=True)
    object_id = sq.Column(sq.Integer, sq.ForeignKey("object.id"), nullable=False)
    distance = sq.Column(sq.Float, nullable=False)

Create a local engine to store data, and remove one if it already exists.

In [5]:
dbfile = 'sessions.db'

try: os.remove(dbfile)
except: pass

In [6]:
engine = sq.create_engine('sqlite:///'+dbfile)

Now, create tables in the database that we defined above.

In [7]:
Base.metadata.create_all(engine)

Check the size of database. It is filled with metadata, but there aren't any entries yet.

In [8]:
!wc -c $dbfile

   12288 sessions.db


## Inserting Data

For all the subequent operations (insert, update, delete, etc.), we are going to use a `sessionmaker` that establishes all conversations with the database. A session represents a 'stagging zone' for all the objects loaded into the database, just like in git. Any changes won't be persisted into the database until they are committed.

Notice that we use ORM module of sqlalchemy to import sessionmaker. ORM stands for Object-relational mapping and is a technique to make relational data compatible with object oriented languages.

In [9]:
from sqlalchemy.orm import sessionmaker

Bind the engine to the metadata of the Base class so that the declaratives can be accessed through a DBSession instance.

In [10]:
Base.metadata.bind = engine

In [11]:
DBSession = sessionmaker(bind=engine)

In [12]:
session = DBSession()

Insert K new objects into the database. Each one has an `ra` and `dec` value, and is understood to be inserted as a row in the "object" table. If the `id` is not supplied, it is set to be the next integer in the sequence, starting from 1 (not zero!).

In [13]:
K = 1000

In [14]:
for i in range(K):
    ra = random.uniform(9.0, 11.0)
    dec = random.uniform(-30.0, -33.0)
    # Make the new object, with id implicitly set to be i+1: 
    new_object = Object(ra=ra, dec=dec)
    # Add it to the transaction list:
    session.add(new_object)

# Carry out the transaction, writing all new records into the database:    
session.commit()

Create a distance record for each object we inserted above.

In [15]:
for i in range(K):
    id = i + 1
    d = random.uniform(40, 60)
    # Set up the distance ready for ingestion, with object_id matched to the object id - note the +1! 
    new_distance = Distance(distance=d, object_id=id)
    session.add(new_distance)
session.commit()

The database file should be bigger now:

In [16]:
!wc -c $dbfile

   61440 sessions.db


## Querying Data

Make a query for the first object in the database and see its attributes.

In [17]:
first_object = session.query(Object).first()
first_object.id, first_object.ra, first_object.dec

(1, 9.105568084282915, -30.583423614884907)

Retrieve all distances in form of a numpy array. 

In [18]:
# Get the complete list of Distance objects:
all_distances = session.query(Distance)
# Unpack into a numpy array:
d_array = np.array([d.distance for d in all_distances])
print d_array[0:20]

[ 40.71564151  51.54109295  50.75183895  48.84103562  58.83610313
  45.00060216  47.8804096   53.45665911  52.73332629  42.40836338
  42.1308022   40.33939697  48.89663337  40.69657645  59.62243992
  58.6645868   55.0334798   43.69656865  57.49751482  48.19858652]


Apply `filter()` method after a query to get specific results.

In [19]:
d = session.query(Distance).filter(Distance.object_id==80).one_or_none()
print d.distance

59.642416899


In [20]:
# Get a list of Distance objects with distance < 40.1 kpc:
small_distances = session.query(Distance).filter(Distance.distance < 40.1).all()
# Unpack into a numpy array:
low_d_array = np.array([d.distance for d in small_distances])
print low_d_array

[ 40.07859662  40.00959001  40.02131525  40.00553749  40.07226964
  40.08372338  40.08191507]


We can also read tables into `pandas` dataframes, for convenience:

In [21]:
import pandas as pd
df = pd.read_sql(session.query(Distance).filter(Distance.distance < 40.1).statement, session.bind)
df.head()

,id,object_id,distance
0,55,55,40.078597
1,166,166,40.009590
2,419,419,40.021315
3,545,545,40.005537
4,790,790,40.072270


## Joining Two Tables

The "distance" table can be joined to the "object" table via the `object_id` foreign key. We can query for objects and their distances in a number of ways. Notice that if we want both object positions and distance we need to query both tables at the same time.

First, let's associate the IDs manually:

In [22]:
for o, d in session.query(Object, Distance).filter(Object.id==Distance.object_id).filter(Distance.distance < 40.1).all():
    print o.id, o.ra, o.dec, d.distance, d.id, d.object_id

55 9.12395708097 -30.6363808189 40.0785966214 55 55
166 9.99104740738 -30.7407458323 40.0095900075 166 166
419 9.44818681499 -30.7924035242 40.0213152496 419 419
545 10.0686130905 -32.9475028787 40.0055374938 545 545
790 10.8253220655 -32.1530747425 40.0722696372 790 790
893 10.4633925917 -32.2714083002 40.0837233819 893 893
964 9.29299441856 -30.6329805629 40.0819150735 964 964


Compare this with the `join` command:

In [23]:
for o, d in session.query(Object, Distance).join(Distance).filter(Distance.distance < 40.1).all():
    print o.id, o.ra, o.dec, d.distance, d.id, d.object_id

55 9.12395708097 -30.6363808189 40.0785966214 55 55
166 9.99104740738 -30.7407458323 40.0095900075 166 166
419 9.44818681499 -30.7924035242 40.0213152496 419 419
545 10.0686130905 -32.9475028787 40.0055374938 545 545
790 10.8253220655 -32.1530747425 40.0722696372 790 790
893 10.4633925917 -32.2714083002 40.0837233819 893 893
964 9.29299441856 -30.6329805629 40.0819150735 964 964


The above query works because we defined `object.id` as a foreign key for the `distance` table. If we had not done this, we could still join the tables manually with the following:

In [24]:
for o, d in session.query(Object, Distance).join(Distance, Object.id==Distance.object_id).filter(Distance.distance < 40.1).all():
    print o.id, o.ra, o.dec, d.distance, d.id, d.object_id

55 9.12395708097 -30.6363808189 40.0785966214 55 55
166 9.99104740738 -30.7407458323 40.0095900075 166 166
419 9.44818681499 -30.7924035242 40.0213152496 419 419
545 10.0686130905 -32.9475028787 40.0055374938 545 545
790 10.8253220655 -32.1530747425 40.0722696372 790 790
893 10.4633925917 -32.2714083002 40.0837233819 893 893
964 9.29299441856 -30.6329805629 40.0819150735 964 964


## Clean up

In [25]:
try: os.remove(dbfile)
except: pass